### Lab: Implementing Stored Procedures, Triggers, and UDFs in Azure Cosmos DB Using Python SDK with a Movie Database

#### Objective:

Participants will gain hands-on experience in using stored procedures, triggers, and user-defined functions (UDFs) in Azure Cosmos DB to efficiently manage a movie database. This includes batch operations, data validation, and custom computations using Python SDK.

### Lab Steps:

### Step 1. Setup Cosmos DB and Environment
Create a Cosmos DB account, database, and container:

- Database: ecommerce-db
- Container: orders with /userId as the partition key.

In [ ]:
from azure.cosmos import CosmosClient, PartitionKey

# Connection details

COSMOS_DB_URI = "https://localhost:8081"
COSMOS_DB_KEY = "C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw=="


# Initialize Cosmos DB client
client = CosmosClient(COSMOS_DB_URI, COSMOS_DB_KEY,connection_verify=False)

# Database and container setup
database_name = 'ecommerce-db'
container_name = 'orders'

# Create database and container if they do not exist
database = client.create_database_if_not_exists(id=database_name)
container = database.create_container_if_not_exists(
    id=container_name,
    partition_key=PartitionKey(path='/userId')
)

print("Connected to Cosmos DB!")

2. **Insert some data**

In [ ]:
# Sample users to insert
users = [
    {"id": "1", "userId": "user1", "name": "Alice", "totalAmount": 120.50},
    {"id": "2", "userId": "user1", "name": "Alice", "totalAmount": 80.00},
    {"id": "3", "userId": "user2", "name": "Bob", "totalAmount": 200.00},
    {"id": "4", "userId": "user2", "name": "Bob", "totalAmount": 150.00},
    {"id": "5", "userId": "user3", "name": "Charlie", "totalAmount": 300.00},
    {"id": "6", "userId": "user3", "name": "Charlie", "totalAmount": 100.00}
]

# Insert the users into the container
for user in users:
    container.upsert_item(user)  # `upsert_item` creates or updates the document

print("Sample users added successfully.")

## Step 2: Stored Procedure for Order Checkout

Stored Procedure: A bulk operation to add multiple items to an order and calculate the total amount.


1. **Define the Stored Procedure**

In [ ]:
stored_proc = {
    'id': 'calculateTotalOrderValue',
    'body': '''
    function calculateTotalOrderValue(userId) {
        var context = getContext();
        var container = context.getCollection();
        var response = context.getResponse();

        // Validate input
        if (!userId) {
            throw new Error("The 'userId' parameter is required.");
        }

        // Query to sum up all order amounts for the given user
        var query = 'SELECT VALUE SUM(c.totalAmount) FROM c WHERE c.userId = @userId';
        var parameters = [{ name: '@userId', value: userId }];

        // Execute the query
        var isAccepted = container.queryDocuments(
            container.getSelfLink(),
            { query: query, parameters: parameters },
            {},
            function (err, documents, responseOptions) {
                if (err) throw new Error("Error: " + err.message);

                // If no results, return 0
                if (documents.length === 0) {
                    response.setBody(0);
                } else {
                    response.setBody(documents[0]);
                }
            }
        );

        if (!isAccepted) {
            throw new Error("Query was not accepted by the server.");
        }
    }
    '''
}

# Deploy the stored procedure
container.scripts.create_stored_procedure(body=stored_proc)
print("Stored procedure 'calculateTotalOrderValue' created successfully.")


2. **Execute the Stored Procedure**:

In [ ]:
# Specify the userId (partition key) for which to calculate total order value
user_id = 'user1'

# Execute the stored procedure
result = container.scripts.execute_stored_procedure(
    sproc='calculateTotalOrderValue',
    partition_key=user_id,  # Partition key
    params=[user_id]        # Pass the userId as a parameter
)

print(f"Total order value for user '{user_id}': {result}")

# Create a Trigger 

### Trigger: Validate and Enhance User Data
This trigger will:

- Validate that the required fields (userId, name, totalAmount) are present in the document.
- Automatically add a createdAt timestamp if it's not provided.

In [ ]:
trigger = {
    'id': 'validateAndEnhanceUserData',
    'body': '''
    function validateAndEnhanceUserData() {
        var context = getContext();
        var request = context.getRequest();
        var user = request.getBody();

        // Validate required fields
        if (!user.userId) {
            throw new Error("Missing field: 'userId' is required.");
        }
        if (!user.name) {
            throw new Error("Missing field: 'name' is required.");
        }
        if (user.totalAmount == null) {
            throw new Error("Missing field: 'totalAmount' is required.");
        }

        // Add a createdAt timestamp if it's missing
        if (!user.createdAt) {
            user.createdAt = new Date().toISOString();
        }

        // Update the request body with the enhanced user object
        request.setBody(user);
    }
    ''',
    'triggerType': 'Pre',  # Pre-trigger
    'triggerOperation': 'Create'  # Trigger only on item creation
}

# Deploy the trigger
container.scripts.create_trigger(body=trigger)
print("Trigger 'validateAndEnhanceUserData' created successfully.")


2. **Test the trigger**

In [ ]:
# Insert a user using the trigger
user = {
    "id": "7",
    "userId": "user4",
    "name": "David",
    "totalAmount": 150.00
    # No createdAt field here; the trigger will add it
}

result = container.create_item(
    body=user,
    pre_trigger_include=['validateAndEnhanceUserData']  # Use the trigger
)

print("User inserted with trigger:", result)


3. **Let's check for missing value**

- Exception is expected because we have intentionally missed the name

In [21]:
from azure.cosmos.exceptions import CosmosHttpResponseError
try:
    # Insert a user using the trigger
    user = {
        "id": "8",
        "userId": "user5",
        "totalAmount": 150.00
        # No createdAt field here; the trigger will add it
    }

    result = container.create_item(
        body=user,
        pre_trigger_include=['validateAndEnhanceUserData']  # Use the trigger
    )

    print("User inserted with trigger:", result)
except CosmosHttpResponseError as e:
    # Extract a nice one-line message from the error
    print(f"Error: {e.message}")

Error: (BadRequest) Message: {"Errors":["Encountered exception while executing Javascript. Exception = Error: Missing field: 'name' is required.\r\nStack trace: Error: Missing field: 'name' is required.\n   at validateAndEnhanceUserData (validateAndEnhanceUserData.js:12:13)\n   at __docDbMain (validateAndEnhanceUserData.js:28:5)\n   at Global code (validateAndEnhanceUserData.js:1:2)"]}
ActivityId: 75541214-e437-4e6f-a21d-31c8ba92da2b, Request URI: /apps/DocDbApp/services/DocDbServer12/partitions/a4cb4958-38c8-11e6-8106-8cdcd42c33be/replicas/1p/, RequestStats: 
RequestStartTime: 2025-01-28T10:58:31.3173081Z, RequestEndTime: 2025-01-28T10:58:31.3217079Z,  Number of regions attempted:1
{"systemHistory":[{"dateUtc":"2025-01-28T10:57:31.8991077Z","cpu":5.977,"memory":4154988.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.3581,"availableThreads":32765,"minThreads":16,"maxThreads":32767},"numberOfOpenTcpConnection":2},{"dateUtc":"2025-01-28T10:57:41.9138273Z","cpu":6.

c:\Users\Navdeep\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


# Deploy the UDF


This UDF will calculate the discounted price for a given item based on its price and a discount percentage.

In [22]:
# Define the UDF
udf = {
    'id': 'calculateDiscount',
    'body': '''
    function calculateDiscount(price, discountPercentage) {
        if (price == null || discountPercentage == null) {
            throw new Error("Both price and discountPercentage are required.");
        }

        if (discountPercentage < 0 || discountPercentage > 100) {
            throw new Error("Discount percentage must be between 0 and 100.");
        }

        return price - (price * discountPercentage / 100);
    }
    '''
}

# Deploy the UDF to the container
container.scripts.create_user_defined_function(body=udf)
print("UDF 'calculateDiscount' created successfully.")


UDF 'calculateDiscount' created successfully.


c:\Users\Navdeep\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


2. **Test the UDF**

In [23]:
# Query using the UDF
query = """
SELECT c.id, c.name, c.totalAmount, udf.calculateDiscount(c.totalAmount, 10) AS discountedPrice
FROM c
"""

# Run the query
results = container.query_items(
    query=query,
    enable_cross_partition_query=True  # Enable cross-partition query if needed
)

# Print the results
print("Query results:")
for item in results:
    print(item)


Query results:
{'id': '1', 'name': 'Alice', 'totalAmount': 120.5, 'discountedPrice': 108.45}
{'id': '2', 'name': 'Alice', 'totalAmount': 80, 'discountedPrice': 72}
{'id': '3', 'name': 'Bob', 'totalAmount': 200, 'discountedPrice': 180}
{'id': '4', 'name': 'Bob', 'totalAmount': 150, 'discountedPrice': 135}
{'id': '5', 'name': 'Charlie', 'totalAmount': 300, 'discountedPrice': 270}
{'id': '6', 'name': 'Charlie', 'totalAmount': 100, 'discountedPrice': 90}
{'id': '7', 'name': 'David', 'totalAmount': 150, 'discountedPrice': 135}


c:\Users\Navdeep\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
